# 根据要求导出`WRF-CMAQ`模拟数据
两大剖面的数据。用matlab存成三维数组，三个维度分别是水平位置、垂直高度和小时时刻，用这样的三维数组去装载各指标，包括**O3浓度、PM2.5浓度、NO2浓度、各过程贡献、各来源贡献**，例如O3浓度的三维数组是CO3（2点位,300米,4月2日13时）=75μg/m3。其中水平位置可以用1~n代表n个经纬度组合，再单独给一个对照表即可，例如2点位=（E 113.6°、N 29.1°）。

---
*@author: Evan*\
*@date: 2023-05-12*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.io import savemat

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../src/')
from findpoint import findpoint
import ModelEvaLib as me

In [2]:
lon1, lat1 = 116.6, 31.1
lon2, lat2 = 117.5, 30.1

lon3, lat3 = 116.1, 29.7
lon4, lat4 = 117.4, 30.9

In [3]:
chemFile = xr.open_dataset('F:/Data/case_anqing/April/April_chem.nc')
metFile  = xr.open_dataset('F:/Data/case_anqing/April/April_met.nc')
isam = xr.open_dataset('F:/Data/case_anqing/April/April_ISAM1.nc')
pa=xr.open_dataset('F:/Data/case_anqing/April/April_PA1.nc')

nlevel = 21
number = 50

lat = chemFile.latitude
lon = chemFile.longitude

O3   = chemFile.O3[:,:nlevel,:,:]
NO2  = chemFile.NO2[:,:nlevel,:,:]
PM25 = chemFile.PM25[:,:nlevel,:,:]
ht   = chemFile.HT[:,:nlevel,:,:]

# mda8   = O3.rolling(time=8).mean().resample({'time':'D'}).max(dim='time')
# NO2avg = NO2.resample({'time':'D'}).mean(dim='time')
# PMavg  = PM25.resample({'time':'D'}).mean(dim='time')
# htavg  = ht.resample({'time':'D'}).mean(dim='time')
# print(mda8.shape)

HADV = pa.HADV_O3[:,:nlevel,:,:]
ZADV = pa.ZADV_O3[:,:nlevel,:,:]
HDIF = pa.HDIF_O3[:,:nlevel,:,:]
VDIF = pa.VDIF_O3[:,:nlevel,:,:]
DDEP = pa.DDEP_O3[:,:nlevel,:,:]
CHEM = pa.CHEM_O3[:,:nlevel,:,:]

AQ  = isam.O3_AQ[:,:nlevel,:,:]
HF  = isam.O3_HF[:,:nlevel,:,:]
TL  = isam.O3_TL[:,:nlevel,:,:]
CZ  = isam.O3_CZ[:,:nlevel,:,:]
JJ  = isam.O3_JJ[:,:nlevel,:,:]
OTH = isam.O3_OTH[:,:nlevel,:,:]
BCO = isam.O3_BCO[:,:nlevel,:,:]
ICO = isam.O3_ICO[:,:nlevel,:,:]

## Line2

In [14]:
# 生成沿两个点连线的坐标
lons2 = np.linspace(lon3,lon4,num=number)
lats2 = np.linspace(lat3,lat4,num=number)

# 选取这些点在数据集中的最接近的网格点的数据
x_index = []
y_index = []

for i in range(number): 
    out_x, out_y = findpoint(lons2[i],lats2[i],O3)
    x_index.append(out_x)
    y_index.append(out_y)

# CHEM
O3_sel   = O3.isel(x=x_index, y=y_index)
NO2_sel  = NO2.isel(x=x_index, y=y_index)
PM25_sel = PM25.isel(x=x_index, y=y_index)
ht_sel    = ht.isel(x=x_index, y=y_index)

# PA
HADV_sel  = HADV.isel(x=x_index, y=y_index)
ZADV_sel  = ZADV.isel(x=x_index, y=y_index)
HDIF_sel  = HDIF.isel(x=x_index, y=y_index)
VDIF_sel  = VDIF.isel(x=x_index, y=y_index)
DDEP_sel  = DDEP.isel(x=x_index, y=y_index)
CHEM_sel  = CHEM.isel(x=x_index, y=y_index)

# ISAM
AQ_sel  = AQ.isel(x=x_index, y=y_index)
HF_sel  = HF.isel(x=x_index, y=y_index)
TL_sel  = TL.isel(x=x_index, y=y_index)
CZ_sel  = CZ.isel(x=x_index, y=y_index)
JJ_sel  = JJ.isel(x=x_index, y=y_index)
OTH_sel = OTH.isel(x=x_index, y=y_index)
BCO_sel = BCO.isel(x=x_index, y=y_index)
ICO_sel = ICO.isel(x=x_index, y=y_index)

In [15]:
diag_O3   = np.zeros([720,21,50])
diag_NO2  = np.zeros([720,21,50])
diag_PM25 = np.zeros([720,21,50])
diag_ht   = np.zeros([720,21,50])
diag_HADV = np.zeros([720,21,50])
diag_ZADV = np.zeros([720,21,50])
diag_HDIF = np.zeros([720,21,50])
diag_VDIF = np.zeros([720,21,50])
diag_DDEP = np.zeros([720,21,50])
diag_CHEM = np.zeros([720,21,50])
diag_AQ   = np.zeros([720,21,50])
diag_HF   = np.zeros([720,21,50])
diag_TL   = np.zeros([720,21,50])
diag_CZ   = np.zeros([720,21,50])
diag_JJ   = np.zeros([720,21,50])
diag_OTH  = np.zeros([720,21,50])
diag_BCO  = np.zeros([720,21,50])
diag_ICO  = np.zeros([720,21,50])

for i in range(50):
    diag_O3[:,:,i]   = O3_sel[:,:,i,i]
    diag_NO2[:,:,i]  = NO2_sel[:,:,i,i]
    diag_PM25[:,:,i] = PM25_sel[:,:,i,i]
    diag_ht[:,:,i]   = ht_sel[:,:,i,i]
    diag_HADV[:,:,i] = HADV_sel[:,:,i,i]
    diag_ZADV[:,:,i] = ZADV_sel[:,:,i,i]
    diag_HDIF[:,:,i] = HDIF_sel[:,:,i,i]
    diag_VDIF[:,:,i] = VDIF_sel[:,:,i,i]
    diag_DDEP[:,:,i] = DDEP_sel[:,:,i,i]
    diag_CHEM[:,:,i] = CHEM_sel[:,:,i,i]
    diag_AQ[:,:,i]   = AQ_sel[:,:,i,i]
    diag_HF[:,:,i]   = HF_sel[:,:,i,i]
    diag_TL[:,:,i]   = TL_sel[:,:,i,i]
    diag_CZ[:,:,i]   = CZ_sel[:,:,i,i]
    diag_JJ[:,:,i]   = JJ_sel[:,:,i,i]
    diag_OTH[:,:,i]  = OTH_sel[:,:,i,i]
    diag_BCO[:,:,i]  = BCO_sel[:,:,i,i]
    diag_ICO[:,:,i]  = ICO_sel[:,:,i,i]

In [16]:
df_line2 = pd.DataFrame(
    index=np.arange(1,51),
    data={'经度':lons2,'纬度':lats2}
)
df_line2.to_excel('D:/Download/sec2.xlsx')

savemat(
    'D:/Download/sec2.mat',
    {
    'height':diag_ht,
    'O3':diag_O3,
    'NO2':diag_NO2,
    'PM25':diag_PM25,
    'HADV':diag_HADV,
    'ZADV':diag_ZADV,
    'HDIF':diag_HDIF,
    'VDIF':diag_VDIF,
    'DDEP':diag_DDEP,
    'CHEM':diag_CHEM,
    'AQ':diag_AQ,
    'HF':diag_HF,
    'TL':diag_TL,
    'CZ':diag_CZ,
    'JJ':diag_JJ,
    'OTH':diag_OTH,
    'BCO':diag_BCO,
    'ICO':diag_ICO
    }
    )